In [1]:
# 사용할 패키지를 import합니다
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic") # window
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

import seaborn as sns
print('pandas', sns.__version__)

import glob

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import scipy.stats as spy
import pickle
import networkx as nx

python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
numpy 1.21.5
pandas 1.4.4
matplotlib 3.5.2
pandas 0.11.2


# ttest

In [2]:
def get_sigGene_by_ttest(df):
    p_df=pd.DataFrame(columns=['tval','pval'])
    for col in df.columns:
        if col=='class':
            continue
        g0=df.loc[df['class']==0,col]
        g1=df.loc[df['class']==1,col]
        t, p = spy.ttest_ind(g0,g1)
        p_df.loc[col]=[t,p]
    #p_df=p_df.sort_values(by='pval')
    return p_df

In [3]:
merged_df=pickle.load(open('../n_stage_pred_BRCA_0804toPC/mid_result/merged_data/BRCA.pickle', 'rb'))
ccle_genes=list(pd.read_table('../n_stage_pred_BRCA_0804toPC/data/ccle_gene_list.txt')['gene'])
merged_df2=merged_df[ccle_genes+['class']]
ttest_res=get_sigGene_by_ttest(merged_df2)
ttest_res

,tval,pval
TSPAN6,1.565221,0.117887
TNMD,0.602310,0.547121
DPM1,-1.548853,0.121773
SCYL3,-0.017212,0.986271
C1orf112,1.703388,0.088845
...,...,...
POLR2J3,0.500243,0.617028
H2BE1,1.312945,0.189540
AL445238.1,0.782834,0.433933
GET1-SH3BGR,NaN,NaN


# epval 0.005 dst 1

In [6]:
def num_of_neighbor_MGs(net1, g1, dst, MGs):
    net2 = nx.generators.ego_graph(net1, g1, radius=dst)
    if g1 in MGs:
        return len(set(MGs)&set(net2.nodes()))-1
    else:
        return len(set(MGs)&set(net2.nodes()))
    
def get_num_of_ngb_MGs(g1):
    if g1 in ppi_net.nodes():
        num_of_ngb_MGs= num_of_neighbor_MGs(ppi_net, g1, 1, MGs)
    else:
        num_of_ngb_MGs=np.nan
    return num_of_ngb_MGs

In [13]:
FS_res=pickle.load(open("../n_stage_pred_BRCA_0804toPC/main_result/sig_res/BRCA.pickle", 'rb'))
mer1=pd.merge(FS_res, ttest_res['pval'], left_index=True, right_index=True).reset_index()
mer1=mer1.rename(columns={'pval':'ttest_pval', 'index':'gene'})
ep0005=mer1.loc[mer1['epval']<0.005].copy()

## attach num of neighbor MGs
ppi_net=pickle.load(open('../n_stage_pred_BRCA_0804toPC/mid_result/PPI_network.pickle', 'rb'))
MGs=list(ep0005['gene'])
ep0005['num_of_ngb_MGs']=ep0005['gene'].map(get_num_of_ngb_MGs)
ep0005

,gene,FS,epval,ttest_pval,num_of_ngb_MGs
0,ZNF676,0.123038,0.000000,0.109192,0.0
1,FAM163B,0.093631,0.000000,0.031966,0.0
2,LDOC1,0.087836,0.000000,0.000104,1.0
3,FAM78B,0.087489,0.000000,0.000007,0.0
4,IRF1,0.086791,0.000000,0.059006,0.0
...,...,...,...,...,...
197,ILDR2,0.013555,0.004896,0.005204,0.0
198,RGS7,0.013544,0.004896,0.002290,0.0
199,DUSP1,0.013543,0.004896,0.184236,1.0
200,CYP4B1,0.013537,0.004902,0.060669,0.0


In [15]:
ep0005.to_csv("../n_stage_pred_BRCA_0804toPC/main_result/ep0005_ttest_and_PPIdst1.csv")
ep0005.loc[ep0005['ttest_pval']>0.1].sort_values(by='num_of_ngb_MGs', ascending=False).head(20)

,gene,FS,epval,ttest_pval,num_of_ngb_MGs
77,ELAVL1,0.020529,0.001634,0.805131,14.0
120,KRT38,0.016625,0.002845,0.670896,5.0
146,UBP1,0.015383,0.003476,0.545890,3.0
40,KRT4,0.028403,0.000689,0.124859,3.0
72,AZGP1,0.021252,0.001495,0.208698,3.0
36,GOLM1,0.030774,0.000562,0.102812,2.0
17,PLK2,0.045346,0.000133,0.168922,2.0
63,THRSP,0.022573,0.001298,0.665089,1.0
170,PWWP3A,0.014410,0.004183,0.394913,1.0
164,CPEB1,0.014741,0.003894,0.284038,1.0


In [17]:
all_node_ppi['num_of_ngb_MGs'].mean()

0.5431106471816284